In [1]:
from inference_mix import *


In [20]:
data = getSingleAndPressureValues()
with open("data/source-era5_date-2022-01-01_res-0.25_levels-13_steps-01.nc", "rb") as f:
    ref_data = xr.load_dataset(f)

In [41]:
# data = data.drop('number').drop("expver")
def drop_extra_coords(data):
    return data.drop('number').drop("expver")

def fix_time(data):
    # Add "datetime" as an extra coordinate with the same values as "time"
    data = data.assign_coords(datetime=("time", data.time.values))
    
    # Calculate timedelta since the first datetime
    first_datetime = data.datetime.values[0]
    data['time'] = (data.datetime - first_datetime).astype('timedelta64[ns]')
    
    return data

def trim_time(data):
    # Select every 6th sample along the time axis
    return data.isel(time=slice(5, None, 6))

data

<xarray.Dataset> Size: 291MB
Dimensions:                       (time: 13, level: 13, lat: 181, lon: 360)
Coordinates:
  * time                          (time) datetime64[ns] 104B 2024-01-01 ... 2...
  * level                         (level) float64 104B 1e+03 925.0 ... 50.0
  * lat                           (lat) float64 1kB 90.0 89.0 ... -89.0 -90.0
  * lon                           (lon) float64 3kB 0.0 1.0 2.0 ... 358.0 359.0
Data variables: (12/14)
    u_component_of_wind           (time, level, lat, lon) float32 44MB nan .....
    v_component_of_wind           (time, level, lat, lon) float32 44MB nan .....
    geopotential                  (time, level, lat, lon) float32 44MB nan .....
    specific_humidity             (time, level, lat, lon) float32 44MB nan .....
    temperature                   (time, level, lat, lon) float32 44MB nan .....
    vertical_velocity             (time, level, lat, lon) float32 44MB nan .....
    ...                            ...
    2m_temperature                (time, lat, lon) float32 3MB ...
    geopotential_at_surface       (time, lat, lon) float32 3MB ...
    land_sea_mask                 (time, lat, lon) float32 3MB ...
    mean_sea_level_pressure       (time, lat, lon) float32 3MB ...
    toa_incident_solar_radiation  (time, lat, lon) float32 3MB ...
    total_precipitation_6hr       (time, lat, lon) float32 3MB nan ... 0.0004373
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-09T11:30 GRIB to CDM+CF via cfgrib-0.9.1...

In [44]:
trim_time(fix_time(data))
ref_data.level

<xarray.DataArray 'level' (level: 13)> Size: 52B
array([  50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
       1000], dtype=int32)
Coordinates:
  * level    (level) int32 52B 50 100 150 200 250 300 ... 600 700 850 925 1000

In [45]:
def reduce_dims(data, ref_data):
    reduced_data = xr.Dataset()
    
    for var in ref_data.data_vars:
        if var in data:
            # Get the dimensions of the variable in ref_data
            ref_dims = ref_data[var].dims
            
            # Start with the original data for this variable
            reduced_var = data[var]
            
            # Adjust dimensions to match ref_data
            for dim in ref_dims:
                if dim in reduced_var.dims:
                    if dim in ref_data.coords:
                        # Interpolate or select to match the reference coordinate
                        if ref_data[dim].dtype.kind in 'iu':  # Integer dtype
                            reduced_var = reduced_var.sel({dim: ref_data[dim]})
                        else:
                            reduced_var = reduced_var.interp({dim: ref_data[dim]})
            
            # Ensure the variable has exactly the same dimensions as in ref_data
            reduced_var = reduced_var.transpose(*ref_dims)
            
            # Add the reduced variable to the new dataset
            reduced_data[var] = reduced_var
    
    return reduced_data

Variable: geopotential_at_surface
  Dimensions in ref_data: ('lat', 'lon')
  Dimensions in data: ('time', 'lat', 'lon')

Variable: land_sea_mask
  Dimensions in ref_data: ('lat', 'lon')
  Dimensions in data: ('time', 'lat', 'lon')

Variable: 2m_temperature
  Dimensions in ref_data: ('batch', 'time', 'lat', 'lon')
  Dimensions in data: ('time', 'lat', 'lon')

Variable: mean_sea_level_pressure
  Dimensions in ref_data: ('batch', 'time', 'lat', 'lon')
  Dimensions in data: ('time', 'lat', 'lon')

Variable: 10m_v_component_of_wind
  Dimensions in ref_data: ('batch', 'time', 'lat', 'lon')
  Dimensions in data: ('time', 'lat', 'lon')

Variable: 10m_u_component_of_wind
  Dimensions in ref_data: ('batch', 'time', 'lat', 'lon')
  Dimensions in data: ('time', 'lat', 'lon')

Variable: total_precipitation_6hr
  Dimensions in ref_data: ('batch', 'time', 'lat', 'lon')
  Dimensions in data: ('time', 'lat', 'lon')

Variable: toa_incident_solar_radiation
  Dimensions in ref_data: ('batch', 'time', 'lat